In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns

# Create Embeddings

In [2]:
batch_size = 1
context_length = 8  
token_dimensions = 5     
input_text = "The narrator lightly suggested to like and subscribe"

x = torch.randn(batch_size, context_length, token_dimensions)
print(x)

tensor([[[ 0.3503, -0.8094,  1.1897,  0.9066, -0.9312],
         [ 0.6576,  0.7094, -0.2212,  2.1024, -0.2668],
         [ 0.9549, -1.2471, -0.1695,  1.2906,  1.3490],
         [ 0.4314, -0.4416,  0.3027, -0.1840,  1.2651],
         [-0.0237, -1.9411,  0.1802, -0.2259, -0.4298],
         [ 1.1312, -1.6308, -0.1835,  1.1038,  0.8153],
         [-0.7144, -0.1110,  0.0270,  0.8478,  0.1992],
         [ 0.1780, -0.1517,  0.4645,  0.8490, -0.5633]]])


# Initialize Q/K/V Matrices

In [8]:
weight_dimensions = 4
W_query = torch.randn(token_dimensions, weight_dimensions)
W_key = torch.randn(token_dimensions, weight_dimensions)
W_value = torch.randn(token_dimensions, weight_dimensions)
print(W_query)

tensor([[ 1.0867, -0.4136,  0.6036,  1.7178],
        [ 0.0404, -1.8306,  0.0147,  2.1013],
        [-2.5174, -0.7959, -0.0230,  1.7240],
        [ 0.4221,  0.0142, -2.1167,  0.0550],
        [-0.9328, -0.0398, -0.8062,  1.8287]])


# 1. Compute Attention Scores

In [11]:
Q = x @ W_query # all the query vectors
K = x @ W_key
V = x @ W_value

In [13]:
# Compute the dot product
scores = Q @ K.transpose(-2, -1)
scores

tensor([[[  4.8644,   2.5855,  -4.5968,  -4.7806,   1.1133,  -2.5458,  -0.9464,
            3.0037],
         [  8.3941,  25.6939,  -4.0427, -13.8892,  -9.3144,  -2.6578,   2.4561,
            9.9588],
         [ 17.8088,  12.0326,  -9.3354, -13.6517,   4.0069,  -3.8266,  -0.0899,
           11.2886],
         [  8.2973,   7.2580,   8.1912,   1.0921,   3.4672,   8.5117,   2.7320,
            4.9497],
         [  0.8175, -24.3504, -19.1097,  -3.1150,  10.4514, -14.1647,  -7.0077,
           -4.2521],
         [ 13.7313,   1.9084, -13.5010, -11.7617,   5.9565,  -7.7946,  -2.1327,
            7.3473],
         [  8.1362,   8.2262, -10.1147, -11.0576,  -0.8412,  -6.5622,  -1.4202,
            6.0795],
         [  3.3365,   6.0871,  -3.7163,  -5.2978,  -1.7852,  -2.4858,  -0.1748,
            3.0998]]])

# 2. Compute Attention Weights

In [24]:
# Compute the attention weights with softmax (scaled by sqrt of weight dimensions)
attention_weights = F.softmax(scores / (weight_dimensions ** 0.5), dim=-1)
attention_weights

tensor([[[5.0919e-01, 1.6294e-01, 4.4918e-03, 4.0973e-03, 7.8046e-02,
          1.2525e-02, 2.7867e-02, 2.0084e-01],
         [1.7505e-04, 9.9943e-01, 3.4877e-07, 2.5374e-09, 2.4992e-08,
          6.9705e-07, 8.9893e-06, 3.8277e-04],
         [9.1306e-01, 5.0840e-02, 1.1647e-06, 1.3457e-07, 9.1932e-04,
          1.8299e-05, 1.1853e-04, 3.5047e-02],
         [2.4862e-01, 1.4786e-01, 2.3577e-01, 6.7756e-03, 2.2218e-02,
          2.7675e-01, 1.5383e-02, 4.6625e-02],
         [8.0112e-03, 2.7450e-08, 3.7717e-07, 1.1214e-03, 9.9007e-01,
          4.4704e-06, 1.6013e-04, 6.3510e-04],
         [9.3925e-01, 2.5436e-03, 1.1464e-06, 2.7355e-06, 1.9253e-02,
          1.9883e-05, 3.3725e-04, 3.8592e-02],
         [4.1254e-01, 4.3151e-01, 4.4907e-05, 2.8026e-05, 4.6348e-03,
          2.6531e-04, 3.4698e-03, 1.4752e-01],
         [1.6151e-01, 6.3894e-01, 4.7500e-03, 2.1541e-03, 1.2475e-02,
          8.7881e-03, 2.7907e-02, 1.4348e-01]]])

In [25]:
sum(attention_weights[0][0])

tensor(1.0000)

# 3. Update the Meaning

In [27]:
context_vectors = attention_weights @ V
context_vectors

tensor([[[ 0.5757, -1.1351,  1.0509,  1.4269],
         [ 0.7062, -2.7945,  1.4685, -0.8050],
         [ 0.4746, -0.9964,  1.4485,  2.5083],
         [ 1.8337, -1.2298, -2.2148, -1.9156],
         [ 1.5598,  0.2405, -2.0945, -0.2572],
         [ 0.4823, -0.8841,  1.3822,  2.6188],
         [ 0.5380, -1.7296,  1.4418,  0.9801],
         [ 0.6207, -2.1095,  1.3281,  0.0518]]])